In [1]:
import os
os.environ["HF_ENDPOINT"] = "https://hf-mirror.com"
%matplotlib inline
import collections
import copy
import time
import math
import inspect
import random
import pickle
import spacy
import GPUtil
import warnings
import tiktoken
import torch
import torchvision
import pandas as pd
import altair as alt
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from dataclasses import dataclass
from contextlib import nullcontext
from transformers import GPT2LMHeadModel
from transformers import pipeline,set_seed
from os.path import exists

from torch import nn
import torch.nn as nn
import torchvision.datasets as dsets
import torchvision.transforms as transforms
import torch.nn.functional as F
from torch.nn.functional import log_softmax, pad
from torch.utils import data
from torch.utils.data import DataLoader, Dataset, Subset
from torch.optim.lr_scheduler import LambdaLR
from torchtext.data.functional import to_map_style_dataset
from torch.utils.data import DataLoader
from torch.utils.data import TensorDataset
from torchtext.vocab import build_vocab_from_iterator
import torchtext.datasets as datasets
from torch.utils.data.distributed import DistributedSampler
import torch.distributed as dist
import torch.multiprocessing as mp
from torch.nn.parallel import DistributedDataParallel as DDP
from torch.distributed import init_process_group, destroy_process_group
# Set to False to skip notebook execution (e.g. for debugging)
warnings.filterwarnings("ignore")
RUN_EXAMPLES = True

# 设置打印选项，禁用科学计数法
torch.set_printoptions(sci_mode=False)

# 设置打印选项，取消显示省略号
torch.set_printoptions(threshold=float('inf'))

In [10]:
# 获取系统中可用的 CUDA 设备（即 GPU）的数量
torch.cuda.device_count()

def try_gpu(i=0):  #@save
    """如果存在，则返回gpu(i)，否则返回cpu()"""
    if torch.cuda.device_count() >= i + 1:
        return torch.device(f'cuda:{i}')
    return torch.device('cpu')

In [19]:
class BicubicLayer(nn.Module):
    def __init__(self, size=None, scale_factor=None, in_channels=None, out_channels=None,
                 kernel_size=None, padding=None, stride=None):
        super(BicubicLayer, self).__init__()
        
        self.size = size
        self.scale_factor = scale_factor
        
        self.conv = nn.Conv2d(in_channels=in_channels, out_channels=out_channels, 
                              kernel_size=kernel_size, padding=padding, stride=stride)
        
        self.elu = nn.ELU(inplace=True)
        
    def forward(self, X):
        Y = F.interpolate(X, size=self.size, scale_factor=self.scale_factor,mode='bicubic', align_corners=False)
        Y = self.conv(Y)
        Y = self.elu(Y)
        return Y

In [20]:
net_1 = BicubicLayer(size=(16, 16), in_channels=2, out_channels=8, kernel_size=2, padding=(0, 0), stride=(1,1))
net_2 = BicubicLayer(size=(32, 32), in_channels=8, out_channels=16, kernel_size=2, padding=(0, 0), stride=1)
net_3 = BicubicLayer(size=(72, 72), in_channels=16, out_channels=32, kernel_size=3, padding=(1, 1), stride=1) 

In [21]:
# 建立整体网络结构
net_block_1 = nn.Sequential(
    net_1,
    net_2,
    net_3,).to(try_gpu())

In [22]:
from torchsummary import summary
summary(net_block_1, input_size=(2,4,4))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1            [-1, 8, 15, 15]              72
               ELU-2            [-1, 8, 15, 15]               0
      BicubicLayer-3            [-1, 8, 15, 15]               0
            Conv2d-4           [-1, 16, 31, 31]             528
               ELU-5           [-1, 16, 31, 31]               0
      BicubicLayer-6           [-1, 16, 31, 31]               0
            Conv2d-7           [-1, 32, 72, 72]           4,640
               ELU-8           [-1, 32, 72, 72]               0
      BicubicLayer-9           [-1, 32, 72, 72]               0
Total params: 5,240
Trainable params: 5,240
Non-trainable params: 0
----------------------------------------------------------------
Input size (MB): 0.00
Forward/backward pass size (MB): 4.19
Params size (MB): 0.02
Estimated Total Size (MB): 4.21
-----------------------------------------------

In [25]:
class Residual(nn.Module):  
    def __init__(self, input_channels, num_channels,
                 use_1x1conv=False, strides=1):
        super().__init__()
        self.conv1 = nn.Conv2d(input_channels, num_channels,
                               kernel_size=3, padding=1, stride=strides) #高宽不变
        self.conv2 = nn.Conv2d(num_channels, num_channels,
                               kernel_size=3, padding=1)
        if use_1x1conv:
            self.conv3 = nn.Conv2d(input_channels, num_channels,
                                   kernel_size=1, stride=strides)
        else:
            self.conv3 = None
        self.bn1 = nn.BatchNorm2d(num_channels)
        self.bn2 = nn.BatchNorm2d(num_channels)
        self.relu = nn.ReLU(inplace=True)

    def forward(self, X):
        Y =  F.relu(self.bn1(self.conv1(X)))  
        Y = self.bn2(self.conv2(Y))         
        if self.conv3:
            X = self.conv3(X)                
        Y += X
        return F.relu(Y)

In [29]:
net_4 = Residual(32,48,use_1x1conv=True)
net_5 = Residual(48,56,use_1x1conv=True)
net_6 = Residual(56,64,use_1x1conv=True)

In [32]:
# 建立整体网络结构
net_block_2 = nn.Sequential(
    net_1,
    net_2,
    net_3,
    net_4,
    net_5,
    nn.MaxPool2d(kernel_size=2,stride=2,padding=0),
    net_6,
    nn.MaxPool2d(kernel_size=2,stride=2,padding=0),
    ).to(try_gpu())

In [33]:
from torchsummary import summary
summary(net_block_2, input_size=(2,4,4))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1            [-1, 8, 15, 15]              72
               ELU-2            [-1, 8, 15, 15]               0
      BicubicLayer-3            [-1, 8, 15, 15]               0
            Conv2d-4           [-1, 16, 31, 31]             528
               ELU-5           [-1, 16, 31, 31]               0
      BicubicLayer-6           [-1, 16, 31, 31]               0
            Conv2d-7           [-1, 32, 72, 72]           4,640
               ELU-8           [-1, 32, 72, 72]               0
      BicubicLayer-9           [-1, 32, 72, 72]               0
           Conv2d-10           [-1, 48, 72, 72]          13,872
      BatchNorm2d-11           [-1, 48, 72, 72]              96
           Conv2d-12           [-1, 48, 72, 72]          20,784
      BatchNorm2d-13           [-1, 48, 72, 72]              96
           Conv2d-14           [-1, 48,

In [35]:
class Flatten_Module(nn.Module):
    def __init__(self):
        super(Flatten_Module, self).__init__()
        self.flatten = nn.Flatten()
        self.linear = nn.Linear(64 * 18 * 18, 400)

    def forward(self, x):

        flattened = self.flatten(x)  # Shape will be [batch_size, 64 * 9 * 9]
        linear_output = self.linear(flattened)  # Shape will be [batch_size, 1200]
        
        # Reshape to desired shape [batch_size, 20, 60]
        reshaped = linear_output.view( -1,20, 20)
        
        return reshaped

In [36]:
net_7 = Flatten_Module()

In [39]:
# 建立整体网络结构
net = nn.Sequential(
    net_1,
    net_2,
    net_3,
    net_4,
    net_5,
    nn.MaxPool2d(kernel_size=2,stride=2,padding=0),
    net_6,
    nn.MaxPool2d(kernel_size=2,stride=2,padding=0),
    net_7
    ).to(try_gpu())

In [40]:
from torchsummary import summary
summary(net, input_size=(2,4,4))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1            [-1, 8, 15, 15]              72
               ELU-2            [-1, 8, 15, 15]               0
      BicubicLayer-3            [-1, 8, 15, 15]               0
            Conv2d-4           [-1, 16, 31, 31]             528
               ELU-5           [-1, 16, 31, 31]               0
      BicubicLayer-6           [-1, 16, 31, 31]               0
            Conv2d-7           [-1, 32, 72, 72]           4,640
               ELU-8           [-1, 32, 72, 72]               0
      BicubicLayer-9           [-1, 32, 72, 72]               0
           Conv2d-10           [-1, 48, 72, 72]          13,872
      BatchNorm2d-11           [-1, 48, 72, 72]              96
           Conv2d-12           [-1, 48, 72, 72]          20,784
      BatchNorm2d-13           [-1, 48, 72, 72]              96
           Conv2d-14           [-1, 48,

In [58]:
# 初始化一个空列表来存储所有张量
all_raw_target = []

for i in range(502):
    raw_target = torch.load(os.path.join(r"C:\Users\ROG\test_v3\5500", f'input-{i+1}.pt'))

    all_raw_target.append(raw_target)


all_raw_target_stack= torch.stack(all_raw_target) 
all_raw_target_stack.shape

# 计算全局均值和标准差
target_mean = all_raw_target_stack.mean()
target_std = all_raw_target_stack.std()
print(f"Global Mean: {target_mean.item()}, Global Standard Deviation: {target_std.item()}")

# 对每个张量进行 Z-score 归一化
target_normalized_tensors = (all_raw_target_stack - target_mean) / target_std 

for i in range(target_normalized_tensors.shape[0]):  # 300 个样本
    # 提取第 i 个 (20, 20) 的张量
    tensor_i = target_normalized_tensors[i]
    
    # 构建保存路径
    output_file = os.path.join(r"C:\Users\ROG\test_v3\5500_normalized", f'normalized_input_{i+1}.pt')
    
    # 保存张量
    torch.save(tensor_i, output_file)
    print(f"Saved normalized_input{i+1}.pt")

Global Mean: 1480488.125, Global Standard Deviation: 2489774.5
Saved normalized_input1.pt
Saved normalized_input2.pt
Saved normalized_input3.pt
Saved normalized_input4.pt
Saved normalized_input5.pt
Saved normalized_input6.pt
Saved normalized_input7.pt
Saved normalized_input8.pt
Saved normalized_input9.pt
Saved normalized_input10.pt
Saved normalized_input11.pt
Saved normalized_input12.pt
Saved normalized_input13.pt
Saved normalized_input14.pt
Saved normalized_input15.pt
Saved normalized_input16.pt
Saved normalized_input17.pt
Saved normalized_input18.pt
Saved normalized_input19.pt
Saved normalized_input20.pt
Saved normalized_input21.pt
Saved normalized_input22.pt
Saved normalized_input23.pt
Saved normalized_input24.pt
Saved normalized_input25.pt
Saved normalized_input26.pt
Saved normalized_input27.pt
Saved normalized_input28.pt
Saved normalized_input29.pt
Saved normalized_input30.pt
Saved normalized_input31.pt
Saved normalized_input32.pt
Saved normalized_input33.pt
Saved normalized_input

In [72]:
torch.load(r"C:\Users\ROG\test_v3\5500_normalized\normalized_input_278.pt")

tensor([[[ 1.3911,  1.3906],
         [ 1.3897,  1.3889]],

        [[-0.5924, -0.5946],
         [-0.5924, -0.5946]]])

In [73]:
torch.load(r"C:\Users\ROG\test_v3\5500\input-278.pt")

tensor([[[4.9439e+06, 4.9428e+06],
         [4.9404e+06, 4.9386e+06]],

        [[5.5000e+03, 3.0843e-01],
         [5.5000e+03, 3.0843e-01]]])